In [4]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, parallel_bulk

In [5]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=["elastic", "xVw89i=t-Be6_eN7-iNS"],
    ssl_assert_fingerprint=(
        "9b9c41a449eedc4ee0b6d4c555fd821213cc4b85755c0121d172c566b9047604"
    )
)

In [6]:
index_name = "podcasts"

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

In [10]:
import json
import os
import time
import orjson

def extract_data(section_data, items, episode, show, index_name):
    return ({
            "_index": index_name,
            "_id": items,
            "_source": {
                'transcript': section_data['transcript'], 
                'confidence': section_data['confidence'], 
                'words'     : section_data['words'],
                'show'      : show,
                'episode'   : episode
            }
        }
    )

def generate_index_data(podcast_path, index_name):
    start = time.time()
    items = 0
    reads = 0
    for dir in os.listdir(podcast_path):
        if dir == ".DS_Store":
            continue
        for letter in os.listdir(podcast_path + dir):
            if letter == ".DS_Store":
                continue
            for show in os.listdir(podcast_path + dir + "/" + letter):
                if show == ".DS_Store":
                    continue
                for episode in os.listdir(podcast_path + dir + "/" + letter + "/" + show):
                    if episode == ".DS_Store":
                        continue
                    path = podcast_path + dir + "/" + letter + "/" + show + "/" + episode
                    # data = json.load(open(path, "r"))
                    with open("large.json", "rb") as f:
                        data = orjson.loads(f.read())

                        reads += 1

                        for section in data['results']:
                            section_data = section['alternatives'][0]
                            if section_data and 'transcript' in section_data:
                                yield extract_data(section_data, items, episode, show, index_name)
                                items += 1

                    if reads % 2000 == 0:
                        end = time.time()
                        print('took', end - start, 'seconds')
                        start = time.time()
    

In [11]:
path = "./podcasts-no-audio-13GB/spotify-podcasts-2020/podcasts-transcripts/"

In [12]:
from collections import deque
deque(generate_index_data(path, index_name), maxlen=0)

FileNotFoundError: [Errno 2] No such file or directory: 'large.json'

In [20]:
pb = parallel_bulk(es, generate_index_data(path), chunk_size=1000, thread_count=4)

In [22]:
deque(pb, maxlen=0)

deque([], maxlen=0)